***
## CSRF-атаки

В корне рабочей директории компьютера или в какой-нибудь папке для временных файлов создайте HTML-файл для экспериментов; для чистоты эксперимента файл не должен лежать в директории проекта **acme_project**. Имя файла может быть любым, мы назовём его *guile.html*.

Запустите проект **acme_project**.

В файле разместите форму:

```html
<!-- C:/Dev/guile.html -->
<html>
  <head>
    <title>Коварство</title>
  </head>
  <body style="padding:40px;">
    <!-- Адрес отправки формы — проект ACME! -->
    <form action="http://127.0.0.1:8000/birthday/">
      <p>
        <label>Имя</label><br>
        <input type="text" name="first_name" maxlength="20" required>
      </p>
      <p>
        <label>Фамилия</label><br>
        <input type="text" name="last_name" maxlength="20">
      </p>
      <p>
        <label>Дата рождения</label><br>
        <input type="date" name="birthday" required>
      </p>
      <button type="submit">Отправить</button>
    </form>
  </body>
</html> 
```

Такая схема атак известна под названием CSRF (англ. cross-site request forgery — «межсайтовая подделка запроса»). CSRF имеет множество вариаций, и некоторые из них успешно применялись злоумышленниками. 

***
## Защита от CSRF

В Django есть встроенная защита от CSRF: она применяется к формам, запросы из которых изменяют состояние данных на сервере (это в первую очередь POST-, DELETE- и PATCH-запросы).

В общих чертах защита реализована так: в HTML-форму встраивается скрытое поле, в котором записан уникальный ключ, — он называется **csrfmiddlewaretoken**; это последовательность букв и цифр, сгенерированная специальным образом.

```html
<form method="post">
  <!-- Вот он, токен в скрытом (type="hidden") поле формы -->
  <input 
    type="hidden" 
    name="csrfmiddlewaretoken" 
    value="WNLJrHCFlC3gEAjra6QZ3oPDyb7ilZCMYp97DQG7II4G7fvhi2Fr9pRlmNWHOzol">
  ...
  <label class="required" for="id_username">Имя пользователя:</label>
  <input type="text" name="username" id="id_username">
  ...
  <input type="submit" value="Войти">  
</form> 
```

Похожий токен сохраняется и в cookies браузера (злоумышленник не может подменить `cookies` — они ему недоступны).

В итоге при запросе из поддельной формы

* в форме не будет скрытого поля с csrfmiddlewaretoken;

* в cookies не будет нужного токена.

Когда на сервер приходит запрос, проверяется наличие токена в cookies и токен из формы; если токены обнаружены — они по определённому алгоритму сверяются между собой; запрос считается настоящим только в том случае, если проверка прошла успешно. 

Если же проверка не пройдена — вернётся ошибка 403 (недостаточно прав для просмотра страницы).

***
## Защита от CSRF в шаблонах

Из форм, сгенерированных классами `Form` и `ModelForm`, мы пока что отправляли только GET-запросы, и Django без проблем принимал и обрабатывал эти запросы. И даже запрос из «посторонней» формы (из файла *guile.html*) был принят и обработан: ведь это был GET-запрос, а такие запросы только получают информацию, но не меняют её на сервере; эти запросы безопасны.

Но в POST-запросах Django всегда ожидает получить csrf-токен. 

Проверим: измените форму в файле *guile.html* так, чтобы она отправляла запрос методом POST, — и отправьте новый запрос. 

```html
<!-- C:/Dev/guile.html -->
<html>
  <head>
    <title>Коварство</title>
  </head>
  <body style="padding:40px;">
    <!-- Указываем метод POST -->
    <form action="http://127.0.0.1:8000/birthday/" method="post">
      <!-- Всё остальное оставляем, как прежде -->
    </form>
  </body>
</html>
```

Ничего не вышло: запрос отклонён; у злоумышленника проблемы: взять уникальный токен негде.

А вот в форму, созданную в Django, встроить csrf-токен можно и нужно. Этот токен не передаётся в объекте формы; нужно добавить тег `{% csrf_token %}` непосредственно в шаблон, в HTML-код формы. 

Всё остальное шаблонизатор Django сделает сам.

```html
...
<!-- Для форм, отправляющих POST-запросы, csrf-токен необходим! -->
<form method="post">  
  {% csrf_token %}
  ...
</form>
... 
```

Обязательно вставляйте этот тег в формы, которые передают данные методом POST: в любом POST-запросе Django ожидает получить csrf-токен, а в ином случае вернёт ошибку 403.